# Portfolio Project

### 1. Experiment Setup

#### Libraries
#### First, import necessary libraries:

In [6]:
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, confusion_matrix

In [7]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

#### Data Preprocessing

In [15]:
# Image preprocessing (resizing, rescaling)
img_size = (224, 224)
batch_size = 32

train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.15)

train_generator = train_datagen.flow_from_directory(
    'path_to_your_data',
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training')

validation_generator = train_datagen.flow_from_directory(
    'path_to_your_data',
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation')

# Class labels
class_labels = list(train_generator.class_indices.keys())

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'path_to_your_data'

### 2. Custom CNN Architecture
#### Define a simple CNN model for image classification:

In [17]:
def build_custom_cnn(input_shape, num_classes):
    model = Sequential()
    
    # Convolutional layers
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    # Fully connected layers
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    
    model.add(Dense(num_classes, activation='softmax'))
    
    # Compile the model
    model.compile(optimizer=Adam(learning_rate=0.001),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model

# Get input shape and number of classes
input_shape = (224, 224, 3)
num_classes = train_generator.num_classes

# Build and compile model
custom_cnn = build_custom_cnn(input_shape, num_classes)

NameError: name 'train_generator' is not defined

#### Training Custom CNN

In [ ]:
history_custom = custom_cnn.fit(train_generator,
                                epochs=10,
                                validation_data=validation_generator)

### 3. Transfer Learning with Pre-Trained Model
#### Use VGG16 for transfer learning:

In [ ]:
def build_transfer_model(input_shape, num_classes):
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)
    
    # Freeze the base model
    for layer in base_model.layers:
        layer.trainable = False
    
    # Add custom layers on top
    x = base_model.output
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.5)(x)
    output = Dense(num_classes, activation='softmax')(x)
    
    model = Model(inputs=base_model.input, outputs=output)
    
    # Compile model
    model.compile(optimizer=Adam(learning_rate=0.001),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model

# Build transfer learning model
transfer_model = build_transfer_model(input_shape, num_classes)

# Train the model
history_transfer = transfer_model.fit(train_generator,
                                      epochs=10,
                                      validation_data=validation_generator)

### 4. Evaluation
#### Custom CNN Evaluation

In [ ]:
# Evaluate custom CNN on validation data
custom_eval = custom_cnn.evaluate(validation_generator)
print(f"Custom CNN Accuracy: {custom_eval[1]*100:.2f}%")

# Predictions and classification report
pred_custom = custom_cnn.predict(validation_generator)
pred_custom_labels = np.argmax(pred_custom, axis=1)
print(classification_report(validation_generator.classes, pred_custom_labels, target_names=class_labels))

# Confusion matrix
cm_custom = confusion_matrix(validation_generator.classes, pred_custom_labels)
plt.figure(figsize=(10, 8))
sns.heatmap(cm_custom, annot=True, fmt='d', cmap='Blues', xticklabels=class_labels, yticklabels=class_labels)
plt.title('Confusion Matrix - Custom CNN')
plt.show()

#### Transfer Learning Evaluation

In [ ]:
# Evaluate transfer model on validation data
transfer_eval = transfer_model.evaluate(validation_generator)
print(f"Transfer Learning Accuracy: {transfer_eval[1]*100:.2f}%")

# Predictions and classification report
pred_transfer = transfer_model.predict(validation_generator)
pred_transfer_labels = np.argmax(pred_transfer, axis=1)
print(classification_report(validation_generator.classes, pred_transfer_labels, target_names=class_labels))

# Confusion matrix
cm_transfer = confusion_matrix(validation_generator.classes, pred_transfer_labels)
plt.figure(figsize=(10, 8))
sns.heatmap(cm_transfer, annot=True, fmt='d', cmap='Blues', xticklabels=class_labels, yticklabels=class_labels)
plt.title('Confusion Matrix - Transfer Learning')
plt.show()